In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# import xgboost as xgb

from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier,\
                             GradientBoostingClassifier, ExtraTreesClassifier,\
                             BaggingClassifier, VotingClassifier)

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Perceptron
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.cross_validation import KFold

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

from time import time

C:\Users\Francisco\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# Set de entrenamiento y Test final

Cargamos los archivos necesarios

In [2]:
#train = pd.read_csv('modelo.csv')
train = pd.read_csv('modelo.csv')
test_final = pd.read_csv('test_final.csv')

Nos guardamos los IDs necesarios para el submit

In [3]:
id_aviso_postulante = test_final['id']

Vamos a agrupar todos las zonas pertenecientes a la provincia de Buenos Aires con un 1, y aquellas que no pertenecen con un 0

In [5]:
# La Plata, GBA Oeste, Buenos Aires (fuera de GBA), Capital Federal, Gran Buenos Aires
buenos_aires =  (train['nombre_zona'] == 11) |\
                (train['nombre_zona'] == 12) |\
                (train['nombre_zona'] == 13) |\
                (train['nombre_zona'] == 14) |\
                (train['nombre_zona'] == 15)

train.loc[(buenos_aires), 'buenos_aires'] = 1
train.loc[(~buenos_aires), 'buenos_aires'] = 0

train.sample(5)

,Unnamed: 0,idaviso,idpostulante,cant_vistas,edad,sexo,orden_estudio,titulo,descripcion,tipo_de_trabajo,nivel_laboral,nombre_zona,nombre_area,nombre_empresa,se_postulo,buenos_aires
1245356,1245356,1112256011,Rzrp961,1.0,30.0,1,6.0,Recepcionista Eventual. Belgrano,"<p>Importante laboratorio ubicado en Belgrano,...",10.0,2.0,15.0,Recepcionista,Suple,1,1.0
406091,406091,1111576563,8YJzkM,1.0,28.0,2,10.0,NaN,NaN,0.0,0.0,0.0,NaN,NaN,0,0.0
807057,807057,1805705,OPe9W,1.0,0.0,0,0.0,NaN,NaN,0.0,0.0,0.0,NaN,NaN,0,0.0
667459,667459,1112480313,A36mNZL,1.0,26.0,1,4.0,NaN,NaN,0.0,0.0,0.0,NaN,NaN,0,0.0
1569025,1569025,1112314476,Y1YGZD,1.0,31.0,1,6.0,NaN,NaN,0.0,0.0,0.0,NaN,NaN,1,0.0


In [6]:
# La Plata, GBA Oeste, Buenos Aires (fuera de GBA), Capital Federal, Gran Buenos Aires
buenos_aires =  (test_final['nombre_zona'] == 11) |\
                (test_final['nombre_zona'] == 12) |\
                (test_final['nombre_zona'] == 13) |\
                (test_final['nombre_zona'] == 14) |\
                (test_final['nombre_zona'] == 15)

test_final.loc[(buenos_aires), 'buenos_aires'] = 1
test_final.loc[(~buenos_aires), 'buenos_aires'] = 0

test_final.sample(5)

,id,idaviso,idpostulante,orden_estudio,edad,sexo,titulo,descripcion,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,nombre_empresa,buenos_aires
77037,77037,1112464306,1bzdBz,4.0,32.0,2.0,Operario Depósito,<p>Importante empresa textil busca incorporar ...,15,10.0,3.0,Oficios y Profesiones,Billabong,1.0
16630,16630,1112175375,aLxEm,4.0,64.0,2.0,Supervisor de Cajas - Microcentro,<p>Si queres formar parte de una empresa basad...,15,10.0,3.0,Ventas,FALABELLA S.A.,1.0
70265,70265,1112461720,Rz6DoY1,9.0,25.0,1.0,EJECUTIVOS FUERZA DE VENTAS PYME PARA IMPORTAN...,<p>Para importante Banco seleccionaremos Ejecu...,15,10.0,3.0,Comercial,RANDSTAD,1.0
53902,53902,1112445546,W9AmRDN,10.0,22.0,2.0,Ingeniero de Procesos - Rosario,"<p style=""""><span style="""">Estamos buscando pa...",15,10.0,3.0,Ingeniería de Procesos,RANDSTAD,1.0
98303,98303,1112505585,8MkmV3,15.0,38.0,2.0,Gestor de Cobranzas - Mora Tardia,"<p align=""center"">¿SOS UN EXPERTO EN COBRANZAS...",15,9.0,3.0,Call Center,MO&PC Collections Argentina S.R.L,1.0


Nos guardamos los features que utilizaremos para los algoritmos de Machine Learning

In [8]:
# Features posibles:
f1 = 'orden_estudio'
f2 = 'edad'
f3 = 'sexo'
f4 = 'titulo' #str
f5 = 'decripcion' #str
f6 = 'nombre_zona'
f7 = 'tipo_de_trabajo'
f8 = 'nivel_laboral'
f9 = 'nombre_area' #str
f10 = 'nombre_empresa' #str
f11 = 'buenos_aires'

features = [f1,f7,f8]

Particionamos el set de entrenamiento para realizar pruebas locales de hiper-parámetros antes de realizar cada submit

In [9]:
test_s = 0.25
random_s = 0

x = np.array(train[features])
y = np.array(train['sepostulo'])

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_s, random_state=random_s)

Generamos el set de entrenamiento para el test final

In [10]:
x_test_final = np.array(test_final[features])

# Algoritmos probados

# 1. KNN

### Valores iniciales para K

In [ ]:
t0 = time()

print('VALORES INICIALES PARA K')

k_valores = [5,10,20,30,40,50]
mejor_k = 0
mejor_precision = 0

for k in k_valores:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(x_train, y_train)
    pred = knn.predict(x_test)
    precision = accuracy_score(y_test, pred)
    print('La precisión para k=', k, 'es:',precision)
    if precision > mejor_precision:
        mejor_precision = precision
        mejor_k = k
        
print('Con k=', mejor_k, 'se obtuvo la mayor precisión:', mejor_precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

VALORES INICIALES PARA K


### Valores grid search para K

In [ ]:
t0 = time()

print('VALORES GRID SEARCH PARA K')

k_valores = []

for i in range(mejor_k-2, mejor_k+3):
    k_valores.append(i)
    
for k in k_valores:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(x_train, y_train)
    pred = knn.predict(x_test)
    precision = accuracy_score(y_test, pred)
    print('La precisión para k =', k, 'es:',precision)
    if precision > mejor_precision:
        mejor_precision = precision
        mejor_k = k
        
print('Con k=', mejor_k, 'se obtuvo la mayor precisión:', mejor_precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

### Predecimos el test final con el mejor k obtenido

In [ ]:
# Creamos el KNN
knn = KNeighborsClassifier(n_neighbors=mejor_k)

# Lo entrenamos con la totalidad del set de datos
knn.fit(x,y)

# Predecimos las postulaciones del set final
pred_final = knn.predict(x_test_final)

### Submit File

In [ ]:
submit = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final})

submit.to_csv('submit_knn.csv', index=False)

submit['sepostulo'].value_counts()

# 2. Random Forest

### Set de entrenamiento

In [11]:
t0 = time()

# Parámetros óptimos
params_optimos_rf = {'n_estimators':18, 'max_features':'sqrt', 'max_depth':15, 'min_samples_split':5,\
                     'min_samples_leaf':2, 'bootstrap':True, 'oob_score':False, 'warm_start':True}

# Creamos el Random Forest
random_forest = RandomForestClassifier(**params_optimos_rf)

# Lo entrenamos con nuestro set de entrenamiento
random_forest.fit(x_train,y_train)

# Predecimos valores de nuestro set de datos
pred = random_forest.predict(x_test)

# Evaluamos la precisión
precision = accuracy_score(y_test, pred)

print("Precisión: ", precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Precisión:  0.549956
Tiempo de ejecución: 20.99003 seconds.


### Test final

In [9]:
# Creamos el random forest
random_forest = RandomForestClassifier(**params_optimos_rf)

# Lo entrenamos con la totalidad del set de datos
random_forest.fit(x,y)

# Predecimos las postulaciones del set final
pred_final = random_forest.predict(x_test_final)

### Submit file

In [10]:
submit = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final})

submit.to_csv('submit_random_forest.csv', index=False)

submit['sepostulo'].value_counts()

1    100000
Name: sepostulo, dtype: int64

# 3. Extra Trees

### Set de entrenamiento

In [70]:
t0 = time()

# Parámetros óptimos
params_optimos_et = {'n_estimators':10, 'max_features':'sqrt', 'max_depth':5, 'min_samples_split':2,\
                                   'min_samples_leaf':2, 'bootstrap':True, 'oob_score':True, 'warm_start':True}

# Creamos el extra_trees
extra_trees = ExtraTreesClassifier(**params_optimos_et)

# Lo entrenamos con nuestro set de entrenamiento
extra_trees.fit(x_train,y_train)

# Predecimos valores de nuestro set de datos
pred = extra_trees.predict(x_test)

# Evaluamos la precisión
precision = accuracy_score(y_test, pred)

print("Precisión: ", precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Precisión:  0.687294
Tiempo de ejecución: 6.77171 seconds.


### Test final

In [71]:
# Creamos el extra trees
extra_trees = ExtraTreesClassifier(**params_optimos_et)

# Lo entrenamos con la totalidad del set de datos
extra_trees.fit(x,y)

# Predecimos las postulaciones del set final
pred_final = extra_trees.predict(x_test_final)

### Submit file

In [72]:
submit = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final})

submit.to_csv('submit_extra_trees.csv', index=False)

submit['sepostulo'].value_counts()

1    84024
0    15976
Name: sepostulo, dtype: int64

# 4. Gaussian Naive Bayes

### Set de entrenamiento

In [26]:
t0 = time()

# Creamos el naive bayes
naive_bayes = GaussianNB()

# Lo entrenamos con nuestro set de entrenamiento
naive_bayes.fit(x_train,y_train)

# Predecimos valores de nuestro set de datos
pred = naive_bayes.predict(x_test)

# Evaluamos la precisión
precision = accuracy_score(y_test, pred)

print("Precisión: ", precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Precisión:  0.686118
Tiempo de ejecución: 0.65951 seconds.


### Test final

In [27]:
# Creamos el naive bayes
naive_bayes = GaussianNB()

# Lo entrenamos con la totalidad del set de datos
naive_bayes.fit(x,y)

# Predecimos las postulaciones del set final
pred_final = naive_bayes.predict(x_test_final)

### Submit file

In [28]:
submit = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final})

submit.to_csv('submit_naive_bayes.csv', index=False)

submit['sepostulo'].value_counts()

1    51352
0    48648
Name: sepostulo, dtype: int64

# 5. Perceptron

### Set de entrenamiento

In [34]:
t0 = time()

# Creamos el perceptron
perceptron = Perceptron()

# Lo entrenamos con nuestro set de entrenamiento
perceptron.fit(x_train,y_train)

# Predecimos valores de nuestro set de datos
pred = perceptron.predict(x_test)

# Evaluamos la precisión
precision = accuracy_score(y_test, pred)

print("Precisión: ", precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Precisión:  0.679952
Tiempo de ejecución: 1.77336 seconds.


### Test final

In [35]:
# Creamos el perceptron
perceptron = Perceptron()

# Lo entrenamos con la totalidad del set de datos
perceptron.fit(x,y)

# Predecimos las postulaciones del set final
pred_final = perceptron.predict(x_test_final)

### Submit file

In [37]:
submit = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final})

submit.to_csv('submit_perceptron.csv', index=False)

submit['sepostulo'].value_counts()

0    99979
1       21
Name: sepostulo, dtype: int64

# 6. SVM

### Set de entrenamiento

In [ ]:
t0 = time()

# Parámetros óptimos
params_optimos_svm = {'C':1.0, 'kernel':'rbf', 'gamma':'auto'}

# Creamos el svm
svm = SVC(**params_optimos_svm)

# Lo entrenamos con nuestro set de entrenamiento
svm.fit(x_train,y_train)

# Predecimos valores de nuestro set de datos
pred = svm.predict(x_test)

# Evaluamos la precisión
precision = accuracy_score(y_test, pred)

print("Precisión: ", precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

### Test final

In [ ]:
# Creamos el svm
svm = SVC(**params_optimos_svm)

# Lo entrenamos con la totalidad del set de datos
svm.fit(x,y)

# Predecimos las postulaciones del set final
pred_final = svm.predict(x_test_final)

### Submit file

In [ ]:
submit = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final})

submit.to_csv('submit_svm.csv', index=False)

submit['sepostulo'].value_counts()

# 7. Adaptive Boosting

### Set de entrenamiento

In [73]:
t0 = time()

# Parámetros óptimos
params_optimos_ab = {'n_estimators':50}

# Creamos el adaptive boosting
ada_boost = AdaBoostClassifier(**params_optimos_ab)

# Lo entrenamos con nuestro set de entrenamiento
ada_boost.fit(x_train,y_train)

# Predecimos valores de nuestro set de datos
pred = ada_boost.predict(x_test)

# Evaluamos la precisión
precision = accuracy_score(y_test, pred)

print("Precisión: ", precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Precisión:  0.686286
Tiempo de ejecución: 42.08794 seconds.


### Test final

In [74]:
# Creamos el adaptive boosting
ada_boost = AdaBoostClassifier(**params_optimos_ab)

# Lo entrenamos con la totalidad del set de datos
ada_boost.fit(x,y)

# Predecimos las postulaciones del set final
pred_final = ada_boost.predict(x_test_final)

### Submit file

In [75]:
submit = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final})

submit.to_csv('submit_ada_boost.csv', index=False)

submit['sepostulo'].value_counts()

1    89747
0    10253
Name: sepostulo, dtype: int64

# 8. Gradient Boosting

### Set de entrenamiento

In [76]:
t0 = time()

# Parámetros óptimos
params_optimos_gb = {'learning_rate':0.1, 'n_estimators':75, 'max_depth':4, 'min_samples_split':2,\
                  'min_samples_leaf':2, 'subsample':1.0, 'max_features':'sqrt', 'warm_start':True}

# Creamos el gradient boosting
gra_boost = GradientBoostingClassifier(**params_optimos_gb)

# Lo entrenamos con nuestro set de entrenamiento
gra_boost.fit(x_train,y_train)

# Predecimos valores de nuestro set de datos
pred = gra_boost.predict(x_test)

# Evaluamos la precisión
precision = accuracy_score(y_test, pred)

print("Precisión: ", precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Precisión:  0.688604
Tiempo de ejecución: 67.73957 seconds.


### Test final

In [77]:
# Creamos el gradient boosting
gra_boost = GradientBoostingClassifier(**params_optimos_gb)

# Lo entrenamos con la totalidad del set de datos
gra_boost.fit(x,y)

# Predecimos las postulaciones del set final
pred_final = gra_boost.predict(x_test_final)

### Submit file

In [78]:
submit = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final})

submit.to_csv('submit_gra_boost.csv', index=False)

submit['sepostulo'].value_counts()

1    85080
0    14920
Name: sepostulo, dtype: int64

# 9. Logistic Regression

### Set de entrenamiento

In [24]:
t0 = time()

# Creamos el logistic regression
log_reg = LogisticRegression()

# Lo entrenamos con nuestro set de entrenamiento
log_reg.fit(x_train,y_train)

# Predecimos valores de nuestro set de datos
pred = log_reg.predict(x_test)

# Evaluamos la precisión
precision = accuracy_score(y_test, pred)

print("Precisión: ", precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Precisión:  0.685206
Tiempo de ejecución: 5.04713 seconds.


### Test final

In [20]:
# Creamos el logistic regression
log_reg = LogisticRegression()

# Lo entrenamos con la totalidad del set de datos
log_reg.fit(x,y)

# Predecimos las postulaciones del set final
pred_final = log_reg.predict(x_test_final)

### Submit file

In [21]:
submit = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final})

submit.to_csv('submit_log_reg.csv', index=False)

submit['sepostulo'].value_counts()

0    51392
1    48608
Name: sepostulo, dtype: int64

# 10. Decision Tree

### Set de entrenamiento

In [45]:
t0 = time()

# Parámetros óptimos
params_optimos_dt = {'criterion':'gini', 'splitter':'best', 'max_depth':None, 'min_samples_split':2,\
                  'min_samples_leaf':5, 'max_features':'sqrt', 'presort':True}

# Creamos el logistic regression
decision_tree = DecisionTreeClassifier(**params_optimos_dt)

# Lo entrenamos con nuestro set de entrenamiento
decision_tree.fit(x_train,y_train)

# Predecimos valores de nuestro set de datos
pred = decision_tree.predict(x_test)

# Evaluamos la precisión
precision = accuracy_score(y_test, pred)

print("Precisión: ", precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Precisión:  0.689064
Tiempo de ejecución: 142.78077 seconds.


### Test final

In [46]:
# Creamos el decision tree
decision_tree = DecisionTreeClassifier(**params_optimos_dt)

# Lo entrenamos con la totalidad del set de datos
decision_tree.fit(x,y)

# Predecimos las postulaciones del set final
pred_final = decision_tree.predict(x_test_final)

### Submit file

In [47]:
submit = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final})

submit.to_csv('submit_decision_tree.csv', index=False)

submit['sepostulo'].value_counts()

1    88926
0    11074
Name: sepostulo, dtype: int64

# 11. Bagging

## 11.1 Con Logistic Regression

### Set de entrenamiento

In [58]:
t0 = time()

# Parámetros óptimos
params_optimos_bag_lr = {'base_estimator':LogisticRegression(), 'n_estimators':100, 'bootstrap':True,\
                  'bootstrap_features':True, 'oob_score':True, 'warm_start':False}

# Creamos el bagging
bagging = BaggingClassifier(**params_optimos_bag_lr)

# Lo entrenamos con nuestro set de entrenamiento
bagging.fit(x_train,y_train)

# Predecimos valores de nuestro set de datos
pred = bagging.predict(x_test)

# Evaluamos la precisión
precision = accuracy_score(y_test, pred)

print("Precisión: ", precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Precisión:  0.686596
Tiempo de ejecución: 364.10332 seconds.


### Test final

In [59]:
# Creamos el bagging
bagging = BaggingClassifier(**params_optimos_bag_lr)

# Lo entrenamos con la totalidad del set de datos
bagging.fit(x,y)

# Predecimos las postulaciones del set final
pred_final = bagging.predict(x_test_final)

### Submit file

In [60]:
submit = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final})

submit.to_csv('submit_bagging_log_reg.csv', index=False)

submit['sepostulo'].value_counts()

1    51633
0    48367
Name: sepostulo, dtype: int64

## 11.2 Con Decision Tree

### Set de entrenamiento

In [48]:
t0 = time()

# Parámetros óptimos
params_optimos_bag_dt = {'base_estimator':DecisionTreeClassifier(**params_optimos_dt), 'n_estimators':10, 'bootstrap':True,\
                  'bootstrap_features':True, 'oob_score':True, 'warm_start':False}

# Creamos el bagging
bagging = BaggingClassifier(**params_optimos_bag_dt)

# Lo entrenamos con nuestro set de entrenamiento
bagging.fit(x_train, y_train)

# Predecimos valores de nuestro set de datos
pred = bagging.predict(x_test)

# Evaluamos la precisión
precision = accuracy_score(y_test, pred)

print("Precisión: ", precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Precisión:  0.689912
Tiempo de ejecución: 431.55534 seconds.


### Test final

In [49]:
# Creamos el bagging
bagging = BaggingClassifier(**params_optimos_bag_dt)

# Lo entrenamos con la totalidad del set de datos
bagging.fit(x,y)

# Predecimos las postulaciones del set final
pred_final = bagging.predict(x_test_final)

### Submit file

In [50]:
submit = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final})

submit.to_csv('submit_bagging_extra_tree.csv', index=False)

submit['sepostulo'].value_counts()

1    93978
0     6022
Name: sepostulo, dtype: int64

# 12. Voting

### Set de entrenamiento

In [14]:
# Creamos los estimadores base que utilizaremos para el voting
estimador_1 = GaussianNB()
estimador_2 = LogisticRegression()
estimador_3 = RandomForestClassifier(**params_optimos_rf)

# Los agregamos a la lista de estimadores
estimadores = []
estimadores.append(('Naive Bayes', estimador_1))
estimadores.append(('Logistic Regression', estimador_2))
estimadores.append(('Random Forest', estimador_3))

# Creamos el voting
voting = VotingClassifier(estimators=estimadores)

# Lo entrenamos con nuestro set de entrenamiento
voting.fit(x_train, y_train)

# Predecimos valores de nuestro set de datos
pred = voting.predict(x_test)

# Evaluamos la precisión
precision = accuracy_score(y_test, pred)

print("Precisión: ", precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Precisión:  0.686504
Tiempo de ejecución: 341.46199 seconds.


C:\Users\Francisco\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


### Test final

In [15]:
# Creamos el voting
voting = VotingClassifier(estimators=estimadores)

# Lo entrenamos con la totalidad del set de datos
voting.fit(x,y)

# Predecimos las postulaciones del set final
pred_final = voting.predict(x_test_final)

C:\Users\Francisco\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


### Submit file

In [16]:
submit = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final})

submit.to_csv('submit_voting.csv', index=False)

submit['sepostulo'].value_counts()

1    92523
0     7477
Name: sepostulo, dtype: int64